# What's happening here?

4_link_video_behavior.ipynb

Run after 1_, 2_, and 3_

Made in environment **PMS_Study**

## TO DO: bekijk file PMS_analysis/2_preparation

Probeer veel met loops en functions te werken om de code overzichtelijk te houden

 * Start with file cleanData.csv (geeft informatie over van welke participant welke files bestaan)
 * Load timings.csv file that Tilia and Sofie made
 * Start loop from the top and use Order and A-B
 * In loop check if file (corresponding CSV) exists in Directory (/10001 or /10002 depedent of A/B): later refered to as behaviordata
 * In loop check if file (corresponding .WEBM) exists in Directory
 
If both exist:
- Check if output of Openface exists in Z:\shares\ghepmk_data\2020_Kappen_PMS\10002\video_features (or /10001 dependent of A/B)
- IF True:
Load this as facedata
- IF False:
Print: "Video file present for file (insert filename) but no openface present)"
Break/skip

Use filename/pptnum to search for right starting time in timings.csv variable
- Find timing for beep.wav in behaviordata. 
- Set new zero point (link between the timings.csv point and this)
- Make mean scores on a few variables (we will expand this later) for the corresponding frames of each of the trials.
- To do this, you should write a function that takes the right points of a trial (start, end, trialnum etc) and save this data to a dataframe

Compute this in a way that you would be able to add it to allPMSdata.cvs

Add it all to a new dataframe

Write to csv

In [1]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd
import math
import time
import json
import statistics

import features as ft

from tqdm import tqdm
from pathlib import Path

import re

In [2]:
# Create all relevant directories
data_dir = Path('Z:/shares/ghepmk_data/2020_Kappen_PMS')
date_dir = "06102021/"
data_all_dir = os.path.join(data_dir, date_dir)

# Load base data
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")
allData = pd.read_csv(data_all_dir + "allPMSdata.csv", encoding = "ISO-8859-1")
timings = pd.read_csv(data_all_dir + "timings.csv", encoding = "ISO-8859-1")
timings = timings.rename(columns={"ï»¿ImageGroup": "ImageGroup"}, index={'ONE': 'Row_1'}) # Rename first column, because was formatted weird

redoOpenface = 0 # Set to 1 if you want to re-do OpenFace processing. For instance if you added features, or changed methods

In [3]:
# Get all OpenFace files
os.chdir(data_dir)
AFiles = glob.glob("10001/video_features/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
BFiles = glob.glob("10002/video_features/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory

# Opening JSON file with all video duration information
f = open(data_all_dir + 'video_durations.json',)
durationsdata = json.load(f)
f.close()

In [10]:
def get_openfacestuff(fileDir):
    print(fileDir)
    if len(indices) > 1:
        print("Too many OpenFace files for ppt: " + str(pptnum) + " at moment " + moment)
    else: # If we get to here - the party really starts
        try:
            math.isnan(allData['AU12'][idx[0]])
        except:
            allData.loc[0, 'OFdone'] = np.nan # In case this column doesn't exist yet, create a cell with nan so the column is created
#             allData["AU12"] = np.nan # In case this column doesn't exist yet, create a column with only nan's so th next if statement always passes

        if math.isnan(allData['OFdone'][idx[0]]) or redoOpenface == 1: # Check if OpenFace values have already been added, of whether manually set at top of script to re do this (so it doesnt happen unneccessarily
            print("Now reading: " + fileDir)
            openface = pd.read_csv(fileDir, encoding = "ISO-8859-1") # Read openface output csv

            # Get corresponding video file duration 
            splits = fileDir.split('/video_features\\')
            fileduration = [value for key, value in durationsdata.items() if splits[0] in key.lower() and splits[1][23:len(splits[1])-4] in key.lower()] # Very weird formatting, but when checking for directory (10001/10002) and only the date (something with curly bracktes messes it up. 
            frameduration = fileduration[0] / len(openface)
            openface.insert (3, "timestamp2", np.arange(0, fileduration[0], frameduration).tolist()[0:len(openface)]) # Add new timestamps right after the first timestamp || We're taking [0:len(openface)] here because due to rounding error, the newly created timestamps might be one longer than th array. 
            videoOffset
            behaviorOffset = allData['startBeep'][idx[0]]
            correctedStarts = allData['startView'][idx] - behaviorOffset # startViews - offSet
            videoStarts = ((correctedStarts + videoOffset) / 1000).tolist() # These will be the timestamps in seconds because / 1000

            correctedEnds = allData['stopView'][idx] - behaviorOffset # startViews - offSet
            videoEnds = ((correctedEnds + videoOffset) / 1000).tolist()
            
            print(videoStarts)

            # Loop over all the trials and get the appropriate values - add to allData
            for trial in range(0,len(videoStarts)):
                vidStartIdx = openface.iloc[(openface['timestamp2']-videoStarts[trial]).abs().argsort()[:1]].index.tolist()[0] # Get start idx in openface
                vidEndIdx = openface.iloc[(openface['timestamp2']-videoEnds[trial]).abs().argsort()[:1]].index.tolist()[0] # Get end idx in openface

                # Create mean value between these idx's and add to big dataframe
                allData.loc[idx[trial], 'AU12'] = openface[' AU12_r'][vidStartIdx:vidEndIdx].mean() 
                allData.loc[idx[trial], 'AU15'] = openface[' AU15_r'][vidStartIdx:vidEndIdx].mean() 
                
                allData.loc[idx[trial], 'arousal'] = ft.compute_arousal(openface[vidStartIdx:vidEndIdx])['mean_Arousal']
                
                # Pupil stuff gave bugs and slowed down the code. Try again later perhaps. 
#                 allData.loc[idx[trial], 'blinkrate'] = ft.compute_blink_rate(openface[vidStartIdx:vidEndIdx])
#                 allData.loc[idx[trial], 'PD_Mean'] = ft.compute_PD_features(openface[vidStartIdx:vidEndIdx])['mean_PD']
#                 allData.loc[idx[trial], 'PD_Std'] = ft.compute_PD_features(openface[vidStartIdx:vidEndIdx])['std_PD']
#                 allData.loc[idx[trial], 'PD_Max'] = ft.compute_PD_features(openface[vidStartIdx:vidEndIdx])['max_PD']
                
                allData.loc[idx[trial], 'headMovement'] = ft.compute_head_motion(openface[vidStartIdx:vidEndIdx])['compound_Motion']

                allData.loc[idx[trial], 'happy'] = ft.compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Happy']
                allData.loc[idx[trial], 'sad'] = ft.compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Sad']
                allData.loc[idx[trial], 'disgust'] = ft.compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Disgust']
                allData.loc[idx[trial], 'anger'] = ft.compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Angry']
                allData.loc[idx[trial], 'scared'] = ft.compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Scared']
                
                allData.loc[idx[trial], 'confidence'] = ft.compute_quality(openface[vidStartIdx:vidEndIdx])['mean_confidence']
            
            allData['OFdone'][idx[0]] = 1

In [11]:
# The BIG loop where all the magic happens

index = 0
for pptnum in tqdm(timings["pptnumber"]): # Loop over all ppt's that had a video available (therefore present in timings.csv)
    if '514' not in str(pptnum) and '583' not in str(pptnum) and '342' not in str(pptnum) and '4888' not in str(pptnum): # '&': This is something that gives errors, '383': Corrupted file, '467': same, '495': same, '502': same

        print("processing pptnum: " + str(pptnum))
        idx = np.where(allData['ID'] == int(pptnum))[0] # Watch out; this will give you ALL indices for this pptnum, so both A and B
        if timings["Exclusion"][index] == 1: # Only continue the loop if there is no Exclusion marked
            print("Exclusion file for ppt: " + str(pptnum) + " at moment: " + timings["ImageGroup"][index])
        else:
            videoOffset = float(timings["StartTime"][index])
            if videoOffset == 0: # if a timing start point was not present, it was set to 0. We don't process those because we cant sync them
                print('start time for this ppt is set to zero')
            else:
                if len(idx) < 1: # Check if occurences in datafile exist for this participant/video file
                    print("No input data found for video file of pptnum: " + str(pptnum) + " with video file " + timings["ImageGroup"][index] )
                else:           
                    moment = timings["ImageGroup"][index]
                    if moment ==  'A':
                        indices = [i for i, s in enumerate(AFiles) if str(pptnum) in s]
                        idx = np.where((allData['ID'] == int(pptnum)) & (allData['Moment'] == 'A'))[0] # So we don't also use the indices that don't apply here right now
                        try:
                            fileDir = AFiles[indices[0]]
                        except:
                            print('this file is not found - video present, but no openface file for ' + str(pptnum))
                        else: 
                            get_openfacestuff(fileDir)
                    elif moment == 'B':
                        indices = [i for i, s in enumerate(BFiles) if str(pptnum) in s]
                        idx = np.where((allData['ID'] == int(pptnum)) & (allData['Moment'] == 'B'))[0] # So we don't also use the indices that don't apply here right now
                        try:
                            fileDir = BFiles[indices[0]]
                        except:
                            print('this file is not found - video present, but no openface file for ' + str(pptnum))
                        else: 
                            get_openfacestuff(fileDir)

                    else:
                        print("Some error with testing moment")
                        break

    index += 1 # Create counter
#     if index > 10:
#         break

 12%|█████████▌                                                                      | 63/525 [00:00<00:00, 621.08it/s]

processing pptnum: 14
Exclusion file for ppt: 14 at moment: B
processing pptnum: 66
Exclusion file for ppt: 66 at moment: B
processing pptnum: 106
No input data found for video file of pptnum: 106 with video file B
processing pptnum: 119
No input data found for video file of pptnum: 119 with video file B
processing pptnum: 123
No input data found for video file of pptnum: 123 with video file B
processing pptnum: 124
No input data found for video file of pptnum: 124 with video file B
processing pptnum: 125
No input data found for video file of pptnum: 125 with video file B
processing pptnum: 126
No input data found for video file of pptnum: 126 with video file B
processing pptnum: 127
start time for this ppt is set to zero
processing pptnum: 129
10002/video_features\video_{ParticipantNo}_129_9-10-2020.csv
processing pptnum: 131
10002/video_features\video_{ParticipantNo}_131_2-11-2020.csv
processing pptnum: 134
10002/video_features\video_{ParticipantNo}_134_16-10-2020.csv
processing pptn

C:\Users\mitch\AppData\Local\Temp/ipykernel_13228/1320438631.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allData['OFdone'][idx[0]] = 1
 19%|███████████████▍                                                                | 101/525 [00:09<00:31, 13.54it/s]

processing pptnum: 293
10002/video_features\video_{ParticipantNo}_293_22-11-2020.csv
Now reading: 10002/video_features\video_{ParticipantNo}_293_22-11-2020.csv
[4.585, 21.357, 38.715, 54.227, 69.667, 91.445, 105.915, 120.539, 134.909, 149.826, 165.433, 180.698, 196.851, 210.769, 225.403, 240.537, 256.338, 271.37, 286.977, 302.483, 322.21, 337.186]
processing pptnum: 294
start time for this ppt is set to zero
processing pptnum: 295
10002/video_features\video_{ParticipantNo}_295_22-11-2020.csv
Now reading: 10002/video_features\video_{ParticipantNo}_295_22-11-2020.csv
[11.771, 29.948, 46.518, 62.609, 78.532, 94.612, 112.109, 130.056, 145.454, 158.675, 174.009, 187.356, 202.014, 214.683, 227.345, 241.715, 254.994, 267.591, 286.942, 301.087, 314.776, 330.332]
processing pptnum: 296
10002/video_features\video_{ParticipantNo}_296_25-11-2020.csv
Now reading: 10002/video_features\video_{ParticipantNo}_296_25-11-2020.csv
[4.932, 22.356, 38.211, 53.849, 68.661, 83.376, 96.172, 110.142, 123.965, 1

 19%|███████████████▍                                                                | 101/525 [00:20<00:31, 13.54it/s]

...

processing pptnum: 676
10001/video_features\video_{ParticipantNo}_676_7-8-2021.csv
Now reading: 10001/video_features\video_{ParticipantNo}_676_7-8-2021.csv
[9.822, 25.522, 39.399, 53.397, 66.415, 79.218, 92.306, 106.109, 118.773, 130.764, 143.842, 155.84, 168.278, 182.527, 195.59, 208.5, 221.458, 233.61, 245.747, 257.555, 272.584, 287.338]


 99%|█████████████████████████████████████████████████████████████████████████████▌| 522/525 [1:07:51<00:40, 13.45s/it]

processing pptnum: 678
start time for this ppt is set to zero
processing pptnum: 679
10001/video_features\video_{ParticipantNo}_679_30-9-2021.csv
Now reading: 10001/video_features\video_{ParticipantNo}_679_30-9-2021.csv
[4.423, 17.309, 31.028, 45.638, 57.956, 71.169, 85.759, 98.917, 110.655, 125.776, 139.78, 154.934, 171.121, 183.67, 196.638, 210.395, 223.878, 236.324, 250.38, 263.307, 275.823, 287.902]


100%|██████████████████████████████████████████████████████████████████████████████| 525/525 [1:08:03<00:00,  7.78s/it]

processing pptnum: 107
No input data found for video file of pptnum: 107 with video file A


In [9]:
print(videoStarts)

NameError: name 'vidStartIdx' is not defined

In [7]:
# os.chdir(os.path.join(data_dir,date_dir))
# allData.to_csv('allPMSdata3.csv')

In [12]:
allData.to_csv(str(data_dir) + '\\' + str(date_dir) + 'allPMSdata.csv')

# Figure out if I can write it without index column, else rename index column



# Where to start when stuff goes wrong

In [8]:
openface = pd.read_csv(fileDir, encoding = "ISO-8859-1") # Read openface output csv

In [9]:
# Get corresponding video file duration 
splits = fileDir.split('/video_features\\')
fileduration = [value for key, value in durationsdata.items() if splits[0] in key.lower() and splits[1][23:len(splits[1])-4] in key.lower()] # Very weird formatting, but when checking for directory (10001/10002) and only the date (something with curly bracktes messes it up. 
frameduration = fileduration[0] / len(openface)
openface.insert (3, "timestamp2", np.arange(0, fileduration[0], frameduration).tolist()[0:len(openface)]) # Add new timestamps right after the first timestamp || We're taking [0:len(openface)] here because due to rounding error, the newly created timestamps might be one longer than th array. 
videoOffset
behaviorOffset = allData['startBeep'][idx[0]]
correctedStarts = allData['startView'][idx] - behaviorOffset # startViews - offSet
videoStarts = ((correctedStarts + videoOffset) / 1000).tolist() # These will be the timestamps in seconds because / 1000

correctedEnds = allData['stopView'][idx] - behaviorOffset # startViews - offSet
videoEnds = ((correctedEnds + videoOffset) / 1000).tolist()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [22]:
idx
# videoStarts
moment

'A'

In [21]:
allData['Moment'][np.where((allData['ID'] == int(pptnum)))[0]]

10208    B
10209    B
10210    B
10211    B
10212    B
10213    B
10214    B
10215    B
10216    B
10217    B
10218    B
10219    B
10220    B
10221    B
10222    B
10223    B
10224    B
10225    B
10226    B
10227    B
10228    B
10229    B
Name: Moment, dtype: object